In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt
import numpy as np
import pandas as pd

import Utilities.Modeling as um
import Utilities.GLOBAL as GV
import Utilities.Utilities as uu
import statsmodels.api as sm

import Models.Soybean_USA_Yield_GA as sy

import warnings # supress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [64]:
f='GA_soy'
i=2762

r = uu.deserialize(f,comment=False)
m = r['model'][i]
stats_model = r['model'][i]

r.keys()

dict_keys(['best_fitness', 'model', 'MAE', 'MAPE', 'fitness', 'corr', 'cv_p', 'cv_r_sq', 'cv_MAE', 'cv_MAPE', 'cv_corr'])

In [66]:
wws=um.var_windows_from_cols(m.params.index)

neg_prec = np.array([(stats_model.params[x]<0 and 'Prec' in x) for x in stats_model.params.index if '-' in x])
pos_prec = np.array([(stats_model.params[x]>0 and 'Prec' in x) for x in stats_model.params.index if '-' in x])

neg_prec_cover = len(um.windows_coverage(wws[neg_prec])[1])
pos_prec_cover = len(um.windows_coverage(wws[pos_prec])[1])

print(neg_prec_cover)
print(pos_prec_cover)

27
0


In [59]:
m.params

const                     -963.949481
year                         0.491359
USA_Sdd30_Jun24-Aug31       -6.099152
USA_Prec_Jul08-Jul09        -0.429221
USA_TempMax_Jun11-Jun24      0.517414
USA_Prec_May20-Jun13        -0.742430
USA_TempMax_Jul20-Jul24      0.520403
dtype: float64

In [60]:
full, actual = um.windows_coverage(wws)
print(type(full),len(full))
print(type(actual), len(actual))

<class 'numpy.ndarray'> 104
<class 'set'> 104


In [61]:
full, actual = um.windows_coverage(wws[neg_prec])
print(type(full),len(full))
print(type(actual), len(actual))

<class 'numpy.ndarray'> 51
<class 'set'> 27


In [43]:
ww=um.windows_from_cols(m.params.index)
full, actual = um.windows_coverage_old(ww)
print(type(full),len(full))
print(type(actual), len(actual))

<class 'numpy.ndarray'> 104
<class 'set'> 104


In [44]:
ww=um.var_windows_from_cols(m.params.index)
full, actual = um.windows_coverage(ww)
print(type(full),len(full))
print(type(actual), len(actual))

<class 'numpy.ndarray'> 104
<class 'set'> 104


In [11]:
scope = sy.Define_Scope()
raw_data = sy.Get_Data_All_Parallel(scope)

In [24]:
# Try replicating the saved results
ww = um.var_windows_from_cols(m.params.index)
model_df = um.extract_yearly_ww_variables(w_df = raw_data['w_w_df_all']['hist'],var_windows= ww)
model_df = pd.concat([raw_data['yield'],model_df], sort=True, axis=1, join='inner')

y_col  ='Yield'
y_df = model_df[[y_col]]


cols= [c for c in m.params.index if c != 'const']
X_df=model_df[cols]

folds = um.folds_expanding(model_df=model_df, min_train_size=10)

cv_score = um.stats_model_cross_validate(X_df, y_df, folds)
cv_score.keys()

dict_keys(['cv_models', 'cv_corr', 'cv_p', 'cv_r_sq', 'cv_y_test', 'cv_y_pred', 'cv_MAE', 'cv_MAPE'])

In [25]:
np.mean(cv_score['cv_p'])

0.00013085595050231662